**ATTENTION!!!**
BE SURE that your runtime is GPU.

In [ ]:
# if you are using this script on server you can specify th visible free cuda device
# the memory usage of devices and their id can be checked with 'nvidia-smi' or 'nvidia-cdl' commands 

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2' # here the id of the device

### Cloning repo from git

This operation requires that you have access to the repo [paradetox](https://github.com/skoltech-nlp/paradetox/).

Generate (or use already existing) your GitHub access token. 

If you need to generate new one, you can use the [instruction](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token).

In [ ]:
!git clone https://<YOUR_TOKEN_HERE>@github.com/skoltech-nlp/paradetox.git

/bin/bash: YOUR_TOKEN_HERE: No such file or directory


### Installing required libs

In [ ]:
!pip install transformers
!pip install python-box
!pip install flair
!pip install fairseq
!pip install sentencepiece
!pip install sacremoses

### Preparing input and args

In [ ]:
from box import Box

In [ ]:
args = Box({
    'batch_size': 32,
    'cola_classifier_path': '/content/drive/MyDrive/style_transfer/cola_classifier',
    'wieting_tokenizer_path': 'sim.sp.30k.model',
    'wieting_model_path': 'sim.pt',
    't1': 75., # this is default value
    't2': 70., # this is default value
    't3': 12. # this is default value
})

In [ ]:
# test example
# TODO: load you data here
# preds: output, inputs: input
preds = [
    'you are fucker',
    'you are fucker',
    'you are fucker'
]

inputs = [
    'you are motherfucker',
    'you are amazing',
    'you are fucker'
]

### Style Transfer Accuracy metric

In [ ]:
from paradetox.evaluation_detox.metric_tools.style_transfer_accuracy import classify_preds
import numpy as np

In [ ]:
accuracy_by_sent = classify_preds(args, preds)
accuracy = np.mean(accuracy_by_sent)
print('\n')
print(accuracy)

### Content Similarity metrics

Aditional data download

In [ ]:
!wget https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
!unzip wieting_similarity_data.zip

--2021-10-12 11:48:37--  https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74066204 (71M) [application/x-zip-compressed]
Saving to: ‘wieting_similarity_data.zip’

wieting_similarity_ 100%[===================>]  70.63M  9.93MB/s    in 9.1s    

2021-10-12 11:48:48 (7.72 MB/s) - ‘wieting_similarity_data.zip’ saved [74066204/74066204]

Archive:  wieting_similarity_data.zip
  inflating: sim.pt                  
  inflating: sim.sp.30k.model        


In [ ]:
from paradetox.evaluation_detox.metric_tools.content_similarity import calc_bleu
from paradetox.evaluation_detox.metric_tools.content_similarity import flair_sim
from paradetox.evaluation_detox.metric_tools.content_similarity import wieting_sim

In [ ]:
bleu = calc_bleu(inputs, preds)
print('\n')
print(bleu)

Calculating BLEU similarity


In [ ]:
emb_sim_stats = flair_sim(args, inputs, preds)
emb_sim = emb_sim_stats.mean()
print('\n')
print(emb_sim)

Calculating flair embeddings similarity
2021-10-12 11:48:52,917 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpgur_rqg4


100%|██████████| 73034624/73034624 [00:09<00:00, 7515429.45B/s]

2021-10-12 11:49:03,483 copying /tmp/tmpgur_rqg4 to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2021-10-12 11:49:03,585 removing temp file /tmp/tmpgur_rqg4


In [ ]:
similarity_by_sent = wieting_sim(args, inputs, preds)
avg_sim_by_sent = similarity_by_sent.mean()
print('\n')
print(avg_sim_by_sent)

Calculating similarity by Wieting subword-embedding SIM model


100%|██████████| 1/1 [00:00<00:00, 25.68it/s]


### Fluency metrics

Additional data preparation:

This part requires CoLa classifier obtained from [here](https://drive.google.com/drive/folders/1p6_3lCbw3J0MhlidvKkRbG73qwmtWuRp). 

IF YOU ARE WORKING IN COLAB: You need to open the link and add the folder `cola_classifier` as an shortcut to your drive (you can allocate shortcut in any place of your drive). The next cell will mount your drive to the current Colab session. You need to specify path of your CoLa shortcut in args `cola_classifier_path`.

IF YOU ARE WORKING ON LOCAL MACHINE: download and unzip the folder from the link and specify the path to the unpacked folder below.

In [ ]:
# if you are working in colab, mount your drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
args = Box({
    'batch_size': 32,
    'cola_classifier_path': '/content/drive/MyDrive/style_transfer/cola_classifier', #TODO: put your path here
    'wieting_tokenizer_path': 'sim.sp.30k.model',
    'wieting_model_path': 'sim.pt',
    't1': 75., # this is default value
    't2': 70., # this is default value
    't3': 12. # this is default value
})

In [ ]:
from paradetox.evaluation_detox.metric_tools.fluency import calc_flair_ppl
from paradetox.evaluation_detox.metric_tools.fluency import calc_gpt_ppl
from paradetox.evaluation_detox.metric_tools.fluency import do_cola_eval

In [ ]:
char_ppl = calc_flair_ppl(preds)
print('\n')
print(char_ppl)

Calculating character-level perplexity


In [ ]:
token_ppl = calc_gpt_ppl(preds)
print('\n')
print(token_ppl)

Calculating token-level perplexity


Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

100%|██████████| 3/3 [00:00<00:00, 14.10it/s]


In [ ]:
cola_stats = do_cola_eval(args, preds)
cola_acc = sum(cola_stats) / len(preds)
print('\n')
print(cola_acc)

Calculating CoLA acceptability stats


1042301B [00:00, 1087865.62B/s]
456318B [00:00, 625495.78B/s]
1it [00:00, 10.30it/s]


### Joint metrics

In [ ]:
from paradetox.evaluation_detox.metric_tools.joint_metrics import *

In [ ]:
gm = get_gm(args, accuracy, emb_sim, char_ppl)
print(gm)
joint = get_j(args, accuracy_by_sent, similarity_by_sent, cola_stats, preds)
print(joint)

### Saving all results

If you obtained all above metrics, you can use this code for pretty results saving:

In [ ]:
import os

In [ ]:
if not os.path.exists('results.md'):
    with open('results.md', 'w') as f:
        f.writelines('| Model | ACC | EMB_SIM | SIM | CharPPL | TokenPPL | FL | GM | J | BLEU |\n')
        f.writelines('| ----- | --- | ------- | --- | ------- | -------- | -- | -- | - | ---- |\n')
        
with open('results.md', 'a') as res_file:
    name = 'TEST' # you can put the specific name of the model
    res_file.writelines(f'{name}|{accuracy:.4f}|{emb_sim:.4f}|{avg_sim_by_sent:.4f}|{char_ppl:.4f}|'
                        f'{token_ppl:.4f}|{cola_acc:.4f}|{gm:.4f}|{joint:.4f}|{bleu:.4f}|\n')